In [ ]:
# 導入必要套件 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import wordcloud
from wordcloud import WordCloud
import re
import nltk
from collections import Counter
nltk.download('stopwords')
from nltk.corpus import stopwords

# 設定繪圖風格
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 5)

In [ ]:
# 讀取資料：注意分隔符
df = pd.read_csv('feedback_sentiment.csv', sep=';')

# 顯示欄位
print("Q1: 有哪些欄位？")
print(df.columns.tolist())

In [ ]:
# Q2：資料清理與特徵工程
print("\nQ2: 資料清理與特徵處理中...")

# 清理資料
df = df.dropna()
df['Date/Time'] = pd.to_datetime(df['Date/Time'], errors='coerce')
df = df.dropna(subset=['Date/Time'])  # 移除轉換失敗的

In [ ]:
# 新增欄位 - 時間元素
df['Month'] = df['Date/Time'].dt.month
df['Day'] = df['Date/Time'].dt.day
df['Hour'] = df['Date/Time'].dt.hour

In [ ]:
# 特徵工程 - 文本分析
df['Total Words'] = df['Text'].apply(lambda x: len(str(x).split()))
df['Total Chars'] = df['Text'].apply(lambda x: len(str(x)))
df['Log Word Count'] = df['Total Words'].apply(lambda x: np.log1p(x))

In [ ]:
# 結果確認
print("\n前五筆資料（新增欄位後）：")
print(df.head())

In [ ]:
# Q3：資料視覺化
print("\nQ3: 視覺化生成中...")

In [ ]:
def plot_sentiment_dist():
    sns.countplot(data=df, x='Sentiment')
    plt.title('1) Sentiment 數量分佈')
    plt.show()

plot_sentiment_dist()

In [ ]:
def plot_sentiment_by_column(col, title):
    sns.countplot(data=df, x=col, hue='Sentiment')
    plt.title(title)
    plt.xticks(rotation=45)
    plt.show()

plot_sentiment_by_column('Source', '2) Source x Sentiment')
plot_sentiment_by_column('Location', '3) Location x Sentiment')

In [ ]:
def plot_distribution_by_sentiment(col, title, kind='hist'):
    if kind == 'hist':
        sns.histplot(data=df, x=col, hue='Sentiment', kde=True)
    elif kind == 'kde':
        sns.kdeplot(data=df, x=col, hue='Sentiment', fill=True)
    plt.title(title)
    plt.show()

plot_distribution_by_sentiment('Confidence Score', '4) Confidence Score x Sentiment')

In [ ]:
plot_sentiment_by_column('Month', '5) Month x Sentiment')
plot_sentiment_by_column('Day', '6) Day x Sentiment')
plot_sentiment_by_column('Hour', '7) Hour x Sentiment')

In [ ]:
plot_distribution_by_sentiment('Total Words', '8) Total Words x Sentiment')
plot_distribution_by_sentiment('Total Chars', '9) Total Chars x Sentiment')

In [ ]:
# 10 & 11: WordCloud by Sentiment
def make_wordcloud(sentiment):
    text = " ".join(df[df['Sentiment'] == sentiment]['Text'].dropna().astype(str))
    wc = WordCloud(stopwords=set(stopwords.words('english')),
                   background_color='white',
                   max_words=100).generate(text)
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'WordCloud: Sentiment = {sentiment}')
    plt.show()

make_wordcloud('Negative')  # 10
make_wordcloud('Positive')  # 11

In [ ]:
# 12 & 13: Top Words 分析
def get_top_words(sentiment, topn=25):
    texts = df[df['Sentiment'] == sentiment]['Text'].dropna().astype(str)
    words = " ".join(texts).lower()
    words = re.findall(r'\b\w+\b', words)
    words = [w for w in words if w not in stopwords.words('english')]
    return Counter(words).most_common(topn)

def plot_top_words(word_freq, title):
    words, freqs = zip(*word_freq)
    sns.barplot(x=list(freqs), y=list(words))
    plt.title(title)
    plt.xlabel("Count")
    plt.show()

neg_words = get_top_words('Negative')
pos_words = get_top_words('Positive')

In [ ]:
plot_top_words(neg_words, "12) Top 25 Negative Words")
plot_top_words(pos_words, "13) Top 25 Positive Words")